In [1]:

%pprint

Pretty printing has been turned OFF



----
# Find Basic Qualifications Section in a File

In [2]:

import sys
sys.path.insert(1, '../py')
from matplotlib.colors import to_hex
from chart_utils import ChartUtilities
from IPython.display import HTML, display

from storage import Storage
s = Storage()

from ha_utils import HeaderAnalysis
ha = HeaderAnalysis(verbose=False)

from scrape_utils import WebScrapingUtilities
wsu = WebScrapingUtilities()
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']

from cypher_utils import CypherUtilities
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

from hc_utils import HeaderCategories
hc = HeaderCategories(cu=cu, verbose=False)

from crf_utils import CrfUtilities
crf = CrfUtilities(ha=ha, hc=hc, cu=cu, verbose=True)
crf.lru.build_isheader_logistic_regression_elements()
crf.lru.build_pos_logistic_regression_elements()


----

In [4]:

%%time
import os

from html_analysis import ElementAnalysis
ea = ElementAnalysis(ha=ha, hc=hc, verbose=False)

import warnings
warnings.filterwarnings('ignore')

CPU times: total: 15.6 ms
Wall time: 15.6 ms


In [5]:

def ellipse_string(unellipsed_str):
    if len(unellipsed_str) > 103:
        ellipsed_str = unellipsed_str[:50] + '…' + unellipsed_str[-50:]
    elif len(unellipsed_str) > 53:
        ellipsed_str = unellipsed_str[:25] + '…' + unellipsed_str[-25:]
    else:
        ellipsed_str = unellipsed_str
    
    return ellipsed_str

In [6]:

def predict_isheader(child_str):
    probs_list = crf.lru.ISHEADER_PREDICT_PERCENT_FIT(child_str)
    idx = probs_list.index(max(probs_list))
    is_header = [True, False][idx]
    
    return is_header


----

In [7]:

%run ../load_magic/dataframes.py

file_name = 'Applied_Scientist_(Remote)_-_Remote_-_Indeed.com_0e94073a1c937bd1.html'
file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
page_soup = get_page_soup(file_path)
row_div_list = page_soup.find_all(name='div', id='jobDescriptionText')
len(row_div_list)

1

In [8]:

child_strs_list = ha.get_navigable_children(row_div_list[0], [])
len(child_strs_list)

25

In [9]:

child_strs_list = cu.get_child_strs_from_file(file_name=file_name)
len(child_strs_list)

25

In [10]:

child_tags_list = ha.get_child_tags_list(child_strs_list)
child_tags_list

['div', 'div', 'div', 'b', 'li', 'li', 'li', 'li', 'li', 'li', 'b', 'li', 'li', 'li', 'li', 'li', 'li', 'li', 'div', 'i', 'i', 'i', 'div', 'i', 'i']

In [14]:

cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)

In [15]:

child_tags_list = cu.get_child_tags_list(child_strs_list, verbose=True)
child_tags_list


                MATCH (np:NavigableParents {navigable_parent: '<i>Note: Yelp does not accept agency resumes. Please do not forward resumes to any recruiting alias or employee. Yelp is not responsible for any fees related to unsolicited resumes.</i>'})<-[s:SUMMARIZES]-(ht:HeaderTags)
                RETURN ht.header_tag AS header_tag;


['div', 'div', 'div', 'b', 'li', 'li', 'li', 'li', 'li', 'li', 'b', 'li', 'li', 'li', 'li', 'li', 'li', 'li', 'div', 'i', 'i', 'i', 'div', 'i', 'i']

In [16]:

is_header_list = ha.get_is_header_list(child_strs_list)
is_header_list

[False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False]

In [17]:

is_header_list = cu.get_is_header_list(child_strs_list)
is_header_list

[None, None, False, True, False, False, False, False, False, False, True, False, False, False, None, False, False, False, False, None, None, False, True, False, False]

In [18]:

# cu.create_h_pickle(verbose=True)
NAVIGABLE_PARENT_IS_HEADER_DICT = s.load_object('NAVIGABLE_PARENT_IS_HEADER_DICT')
for i, (is_header, child_str) in enumerate(zip(is_header_list, child_strs_list)):
    print(i, NAVIGABLE_PARENT_IS_HEADER_DICT.get(child_str), is_header, ellipse_string(child_str))

0 False None <div>Are you intrigued by data? Yelp has hundreds …fety / spam detection, and causal inference.</div>
1 False None <div>Yelp engineering culture is driven by our val…d having fun in a collaborative environment.</div>
2 False False <div>Sound exciting? Keep reading.</div>
3 True True <b>Where You Come In:</b>
4 False False <li>Identify and own challenging problems, form te…theses, and drive significant business impact</li>
5 False False <li>Lead the design and analysis of experiments or…usal and predictive models to test your ideas</li>
6 False False <li>Collaborate with product and engineering to af…stems and provide intelligence to other teams</li>
7 False False <li>Communicate your conc…ical audiences alike</li>
8 False False <li>Keep the team and our projects current on new …rs and attending conferences and local events</li>
9 False False <li>Productionize and aut…thin Python services</li>
10 True True <b>What it Takes to Succeed:</b>
11 False False <li>Experience with 

In [19]:

feature_dict_list = hc.get_feature_dict_list(child_tags_list, is_header_list, child_strs_list)
feature_dict_list[1]

{'initial_tag': 'div', 'is_header': None, 'is_task_scope': None, 'is_minimum_qualification': None, 'is_preferred_qualification': None, 'is_legal_notification': None, 'is_job_title': None, 'is_office_location': None, 'is_job_duration': None, 'is_supplemental_pay': None, 'is_educational_requirement': None, 'is_interview_procedure': None, 'is_corporate_scope': None, 'is_posting_date': None, 'is_other': None, 'child_str': '<div>Yelp engineering culture is driven by our values: we’re a cooperative team that encourages individual authenticity and “unboring” solutions to problems. We enable all new team members to deploy working code their first week, and your impact will only grow from there with the support of your manager, mentor, and team. At the end of the day, we’re all about helping our users, growing as engineers and scientists, and having fun in a collaborative environment.</div>'}

In [20]:

feature_dict_list = cu.get_feature_dict_list(child_tags_list, child_strs_list, verbose=False)
feature_dict_list[1]

{'initial_tag': 'div', 'child_str': '<div>Yelp engineering culture is driven by our values: we re a cooperative team that encourages individual authenticity and unboring solutions to problems. We enable all new team members to deploy working code their first week, and your impact will only grow from there with the support of your manager, mentor, and team. At the end of the day, we re all about helping our users, growing as engineers and scientists, and having fun in a collaborative environment.</div>'}

In [21]:

feature_tuple_list = [hc.get_feature_tuple(feature_dict, crf.lru.pos_lr_predict_single) for feature_dict in feature_dict_list]

In [22]:

crf_list = crf.CRF.predict_single(crf.sent2features(feature_tuple_list))
crf_list

['O-TS', 'O-CS', 'O-CS', 'H-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'H-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-PQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-OL', 'O-LN', 'O-LN', 'O-LN', 'H-LN', 'O-LN', 'O-IP']

In [23]:

from matplotlib.colors import to_hex
from IPython.display import HTML, display
from section_utils import SectionUtilities

su = SectionUtilities(s=s, ha=ha, cu=cu, verbose=False)

# Make an RGB dictionary of all the parts-of-speech symbols
rgba_dict = su.get_pos_color_dictionary()

html_str = ''
for pos, rgba in rgba_dict.items():
    hex_str = to_hex(rgba, keep_alpha=True)
    html_str += f'<span style="color:{hex_str};">{pos}</span>'
    if pos == 'H-O':
        html_str += '<br />'
    else:
        html_str += '&nbsp;'
display(HTML(html_str))

In [26]:

html_str = ''
indices_list = su.get_section(crf_list)
for i, (child_str, pos_symbol) in enumerate(zip(child_strs_list, crf_list)):
    rgba = rgba_dict[pos_symbol]
    hex_str = to_hex(rgba, keep_alpha=True)
    if i == min(indices_list):
        html_str += '<hr />'
    child_str = su.append_pos_symbol(child_str, pos_symbol, use_explanation=True)
    html_str += f'{i+0}) <span style="color:{hex_str};">{child_str}</span><br />'
    if i == max(indices_list):
        html_str += '<hr />'
display(HTML(html_str))

In [31]:

# (10, 11)
idx = 16
child_str = child_strs_list[idx]
child_str = cu.clean_text(child_str)
print(child_str)

<li>The motivation to develop deep product and business knowledge and to connect abstract modeling and analysis tasks with business value</li>


In [32]:

cypher_str = f'''
    MATCH (np:NavigableParents {{navigable_parent: '{child_str}'}})
    SET
        np.is_header = 'False',
        np.is_task_scope = 'False',
        np.is_minimum_qualification = 'True',
        np.is_preferred_qualification = 'False',
        np.is_legal_notification = 'False',
        np.is_job_title = 'False',
        np.is_office_location = 'False',
        np.is_job_duration = 'False',
        np.is_supplemental_pay = 'False',
        np.is_educational_requirement = 'False',
        np.is_interview_procedure = 'False',
        np.is_corporate_scope = 'False',
        np.is_posting_date = 'False',
        np.is_other = 'False'
    RETURN
        np.navigable_parent AS navigable_parent,
        np.is_header AS is_header,
        np.is_task_scope AS is_task_scope,
        np.is_minimum_qualification AS is_minimum_qualification,
        np.is_preferred_qualification AS is_preferred_qualification,
        np.is_legal_notification AS is_legal_notification,
        np.is_job_title AS is_job_title,
        np.is_office_location AS is_office_location,
        np.is_job_duration AS is_job_duration,
        np.is_supplemental_pay AS is_supplemental_pay,
        np.is_educational_requirement AS is_educational_requirement,
        np.is_interview_procedure AS is_interview_procedure,
        np.is_corporate_scope AS is_corporate_scope,
        np.is_posting_date AS is_posting_date,
        np.is_other AS is_other;'''
# print(cypher_str)
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
row_objs_list

[{'navigable_parent': '<li>The motivation to develop deep product and business knowledge and to connect abstract modeling and analysis tasks with business value</li>', 'is_header': 'False', 'is_task_scope': 'False', 'is_minimum_qualification': 'True', 'is_preferred_qualification': 'False', 'is_legal_notification': 'False', 'is_job_title': 'False', 'is_office_location': 'False', 'is_job_duration': 'False', 'is_supplemental_pay': 'False', 'is_educational_requirement': 'False', 'is_interview_procedure': 'False', 'is_corporate_scope': 'False', 'is_posting_date': 'False', 'is_other': 'False'}]

In [26]:

cypher_str = f'''
    MATCH (np:NavigableParents {{navigable_parent: '{child_str}'}})
    RETURN
        np.navigable_parent AS navigable_parent,
        np.is_header AS is_header,
        np.is_task_scope AS is_task_scope,
        np.is_minimum_qualification AS is_minimum_qualification,
        np.is_preferred_qualification AS is_preferred_qualification,
        np.is_legal_notification AS is_legal_notification,
        np.is_job_title AS is_job_title,
        np.is_office_location AS is_office_location,
        np.is_job_duration AS is_job_duration,
        np.is_supplemental_pay AS is_supplemental_pay,
        np.is_educational_requirement AS is_educational_requirement,
        np.is_interview_procedure AS is_interview_procedure,
        np.is_corporate_scope AS is_corporate_scope,
        np.is_posting_date AS is_posting_date,
        np.is_other AS is_other;'''
# print(cypher_str)
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
row_objs_list

[{'navigable_parent': '<li>Communicate your conclusions to technical and non-technical audiences alike</li>', 'is_header': 'False', 'is_task_scope': None, 'is_minimum_qualification': None, 'is_preferred_qualification': None, 'is_legal_notification': None, 'is_job_title': None, 'is_office_location': None, 'is_job_duration': None, 'is_supplemental_pay': None, 'is_educational_requirement': None, 'is_interview_procedure': None, 'is_corporate_scope': None, 'is_posting_date': None, 'is_other': None}]